In [3]:
#!import config/Settings.cs 
#r "nuget: Microsoft.SemanticKernel, 1.11.1"

using System.Collections.Generic;
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Installed Packages Microsoft.SemanticKernel, 1.11.1

In [4]:
public class Item{
    public string name { get; set;}
    public double cost { get; set;}
    public string category { get; set;}

    public Item(string name, double cost, string category = ""){
        this.name = name;
        this.cost = cost;
        this.category = category;
    }

    public override string ToString() => name + ": $" + cost + " - " + category; 
}

public class Receipt{
    public string store { get; set;}
    public List<Item> items { get; set;}
    public DateTime date { get; set;}

    public Receipt(string store, List<Item> items, DateTime date){
        this.store = store;
        this.items = items;
        this.date = date;
    }

    public override string ToString(){
        string printItems = string.Join(", ", items);
        return $"{store} - {date} : \n {printItems}";
    }
}

public class BudgetingApp
{
    public List<Receipt> receipts;
    public List<string> categories;
    Microsoft.SemanticKernel.Kernel kernel;
    string budgetingPluginDirectoryPath;
    Microsoft.SemanticKernel.KernelPlugin budgetingPluginFunctions;

    public BudgetingApp(List<string> categories = null, List<Receipt> receipts = null){
        this.receipts = receipts ?? new List<Receipt>();
        this.categories = categories ?? new List<string>();

        //Build kernel
        var builder = Kernel.CreateBuilder();
        var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

        // Configure AI service credentials used by the kernel
        if (useAzureOpenAI)
            builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
        else
            builder.AddOpenAIChatCompletion(model, apiKey, orgId);

        kernel = builder.Build();

        // BudgetingPlugin directory path
        budgetingPluginDirectoryPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "BudgetingPlugin");

        // Load the BudgetingPlugin from the Plugins Directory
        budgetingPluginFunctions = kernel.ImportPluginFromPromptDirectory(budgetingPluginDirectoryPath);
    }

    public void sortReceipts(){
        for (int r = 0; r < receipts.Count; r++) {
            sortReceipt(receipts[r]);
        }
    }

    public void addCategory(string category){
        categories.Add(category);
        sortReceipts();
    }

    public void addReceipt(Receipt receipt){
        sortReceipt(receipt);
        receipts.Add(receipt);
    }

    void sortReceipt(Receipt receipt){
        for (int i = 0; i < receipt.items.Count; i++)
        {   
                receipt.items[i].category = catagorizeItem(receipt.items[i]).Result;
        }
    }

    async Task<string> catagorizeItem(Item item){
        var arguments = new KernelArguments();
        arguments["categories"] = string.Join(", ", categories);
        arguments["item"] = item.name;
        string category = (await kernel.InvokeAsync(budgetingPluginFunctions["SortItems"], arguments)).ToString();
        return item.category = category;
    }

    public void printSpendingCatagories(){
        Dictionary<string, List<Item>> categorizeItems = new Dictionary<string, List<Item>>();
        Dictionary<string, double> categoryCosts = new Dictionary<string, double>();
        
        foreach(string c in categories){
            categorizeItems.Add(c, new List<Item>());
            categoryCosts.Add(c, 0.0);
        }
        foreach(Receipt r in receipts){
            foreach(Item i in r.items){
                categorizeItems[i.category].Add(i);
                categoryCosts[i.category] += i.cost;
            }
        }
        foreach(string c in categories){
            Console.WriteLine(c + " - "+ categoryCosts[c]);
            Console.WriteLine("----------------------------");
            Console.WriteLine(string.Join(", ", categorizeItems[c]) + "\n");
        }
    }
}

In [5]:
List<Item> targetItems = new List<Item>();
List<Item> QFCItems = new List<Item>();
targetItems.Add(new Item("Phone", 1005.50 ));
targetItems.Add(new Item("Lemons", 4.99 ));
QFCItems.Add(new Item("Milk", 2.99 ));
QFCItems.Add(new Item("Chips", 1.99 ));
QFCItems.Add(new Item("Celery", 4.58 ));

Receipt targetReceipt = new Receipt("Target", targetItems, new DateTime(2024, 9, 17));
Receipt QFCReceipt = new Receipt("QFC", QFCItems, new DateTime(2024, 9, 18));

List<string> tempCategories = new List<string>();
tempCategories.Add("Groceries");
tempCategories.Add("Electronics");

BudgetingApp budgetingApp = new BudgetingApp(tempCategories);

In [6]:
Console.WriteLine("The original QFC receipt before sorting");
Console.WriteLine(QFCReceipt);

budgetingApp.addReceipt(targetReceipt);
budgetingApp.addReceipt(QFCReceipt);

Console.WriteLine("The QFC receipt after adding it to the budget app and sorting");
Console.WriteLine(QFCReceipt);

Console.WriteLine("Spendings");
budgetingApp.printSpendingCatagories();

//Adding a category recategorizes the items
budgetingApp.addCategory("Junk food");

Console.WriteLine("Spendings after adding a Junk Food category");
budgetingApp.printSpendingCatagories();

The original QFC receipt before sorting
QFC - 9/18/2024 12:00:00 AM : 
 Milk: $2.99 - , Chips: $1.99 - , Celery: $4.58 - 
The QFC receipt after adding it to the budget app and sorting
QFC - 9/18/2024 12:00:00 AM : 
 Milk: $2.99 - Groceries, Chips: $1.99 - Groceries, Celery: $4.58 - Groceries
Spendings
Groceries - 14.55
----------------------------
Lemons: $4.99 - Groceries, Milk: $2.99 - Groceries, Chips: $1.99 - Groceries, Celery: $4.58 - Groceries

Electronics - 1005.5
----------------------------
Phone: $1005.5 - Electronics

Spendings after adding a Junk Food category
Groceries - 12.56
----------------------------
Lemons: $4.99 - Groceries, Milk: $2.99 - Groceries, Celery: $4.58 - Groceries

Electronics - 1005.5
----------------------------
Phone: $1005.5 - Electronics

Junk food - 1.99
----------------------------
Chips: $1.99 - Junk food

